In [1]:
import os

In [2]:
%pwd

'/Users/songfxu/Documents/study/MLOps-End2End/research'

In [3]:
os.chdir("../")
%pwd

'/Users/songfxu/Documents/study/MLOps-End2End'

In [4]:
from pydantic import BaseModel
from pathlib import Path

class DataTransformationConfig(BaseModel):
    root_dir: Path
    data_path: Path

In [10]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories

class DataTransformationMgr:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH, schema_filepath = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path= config.data_path
        )
        return data_transformation_config

In [7]:
import os
from src.datascience import logger
from sklearn.model_selection import train_test_split
import pandas as pd

In [12]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
    
    def train_test_splitting(self):
        data = pd.read_csv(self.config.data_path)

        train, test = train_test_split(data)

        train.to_csv(os.path.join(self.config.root_dir, "train.csv"), index = False)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"), index = False)

        logger.info("Split data into training and test sets")
        logger.info(train.shape)
        logger.info(test.shape)

In [13]:
try:
    mgr = DataTransformationMgr()
    config = mgr.get_data_transformation_config()
    DataTransformation(config=config).train_test_splitting()
except Exception as e:
    raise e

[2025-11-17 09:52:16,508: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-11-17 09:52:16,509: INFO: common: yaml file: params.yaml loaded successfully]
[2025-11-17 09:52:16,511: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-11-17 09:52:16,512: INFO: common: Created directory at: artifacts]
[2025-11-17 09:52:16,513: INFO: common: Created directory at: artifacts/data_transformation]
[2025-11-17 09:52:16,526: INFO: 3627374238: Split data into training and test sets]
[2025-11-17 09:52:16,526: INFO: 3627374238: (1199, 12)]
[2025-11-17 09:52:16,526: INFO: 3627374238: (400, 12)]
